### Parsing Flats:

##### Description:

In this project, I want to combine several components at once, namely:
- Parse data from a website about property sales;
- Visualize various metrics that would be interesting to track;
- Systematize this entire cycle.

Roughly, this project consists of the following parts:
- Parsing -> sending data to the database -> data visualization, with all of this happening in a systematic way.

This notebook will contain the code and thoughts throughout my work. The technologies I will use are:
- Python and libraries;
- Clickhouse;
- Superset;
- Airflow;
- Docker.

In [ ]:
import pandas as pd
import requests
from hyper.contrib import HTTP20Adapter
from bs4 import BeautifulSoup as bs
from time import sleep
from airflow.models import Variable
from datetime import datetime, timedelta
from clickhouse_driver import Client
import telegram
import emoji # For the special mission
import warnings
warnings.filterwarnings("ignore")

# Parameters for Airflow
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=3),
    'start_date': datetime(2022, 9, 3),
}
schedule_interval = "00 22 * * 4"

# Configs for Telegram Bot and Clickhouse
client = Client(host=Variable.get("CLICKHOUSE_HOST"))
my_token = Variable.get("TG_TOKEN")
bot = telegram.Bot(token=my_token)
chat_id = Variable.get("CHAT_ID_MOSCOW_FLATS")

What I will parse:
1. The URL;
2. Price per square meter;
3. Cost per square meter;
4. Total apartment price;
5. Distance from the metro.

Warning:
- The data is very messy during collection, so it has to be cleaned at every stage.

In [ ]:
def parce_vacancies():
    lst_links, lst_square, lst_price, lst_subway, lst_description, lst_minutes = [], [], [], [], [], []

    for page in range(1, 99):
        print(emoji.emojize(f'Parsing {page} page :monkey:'))
        full_url = f"https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p={page}"
        source = requests.Session()
        source.mount('https://', HTTP20Adapter())#adapter so that the server does not perceive us as a bot
        response = source.get(full_url)
        sleep(7)
        response.encoding = 'utf-8'
        soup = bs(response.text, 'lxml')

        all_flats = soup.findAll("div", class_="iva-item-content-rejJg")

        for flat in all_flats:

            # Link to the apartment:
            var_link = flat.find("a",
                                 "link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH")
            if var_link is not None:
                local_var_link = "https://www.avito.ru" + var_link.get("href")
                lst_links.append([local_var_link])
            else:
                lst_links.append([None])

            # Price per square meter:
            var_square = flat.find("span",
                                   class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL")
            if var_square is not None:
                var = flat.find("span",
                                class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL").text
                correct_number_square_price = ""
                for var_square_circle in var:
                    if var_square_circle.isdigit():
                        correct_number_square_price += var_square_circle
                # We remove the last digit since the value there is squared:
                correct_number_square_price = correct_number_square_price.replace(correct_number_square_price[-1], "")
                lst_square.append(correct_number_square_price)
            else:
                lst_square.append(None)

            # Full number
            var_full_price = flat.find("span", class_="price-text-_YGDY text-text-LurtD text-size-s-BxGpL")
            if var_full_price is not None:
                full_price = var_full_price.text
                correct_number_full_price = ""
                # We leave only the number:
                for var_full_price_circle in full_price:
                    if var_full_price_circle.isdigit():
                        correct_number_full_price += var_full_price_circle
                lst_price.append(correct_number_full_price)
            else:
                lst_price.append(None)

            # Metro station
            var_subway = flat.find('div', class_="geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL")
            if var_subway is not None:
                subway_name = var_subway.text
                subway_full = ""
                for var_subway_circle in subway_name:
                    if var_subway_circle.isalpha() or var_subway_circle == " " or var_subway_circle == "-":
                        subway_full += var_subway_circle
                    else:
                        break
                # I write if for those cases when the name of the metro includes from and to:
                if (subway_full[-1] == "т" and subway_full[-2] == "о") or (
                        subway_full[-1] == "о" and subway_full[-2] == "д"):
                    subway_full = subway_full[0:-2]
                lst_subway.append(subway_full)
            else:
                lst_subway.append(None)

            # Distance to metro:
            var_minutes = flat.find("span", class_="geo-periodSection-bQIE4")
            if var_minutes is not None:
                var_minutes_text = var_minutes.text
                var_minutes_full = ""
                for var_minute_circle in var_minutes_text:
                    if var_minute_circle == '–' or var_minute_circle.isdigit():
                        var_minutes_full += var_minute_circle
                lst_minutes.append(var_minutes_full)
            else:
                lst_minutes.append(None)

    full_df = pd.DataFrame(
        data={"square_price": lst_square, "full_price": lst_price, "subway": lst_subway, "links": lst_links,
              'destination_from_nearest_subway': lst_minutes})

    return full_df

In [ ]:
full_df = parce_vacancies()

I'll create a copy of df so that if I transform any column, I can return to the original dataframe:

In [ ]:
full_df_test = full_df.copy()
full_df_test.head()

Next, I will filter the data frame from empty values, change the data types, and correct all the shortcomings with links:

In [ ]:
def filter_df(df):
    df = df.drop_duplicates(
        subset="links")  # Sometimes duplicates appear because the site 'dynamically' moves to the next page
    df = df.dropna()
    df[["square_price", "full_price"]] = df[["square_price", "full_price"]].astype("int64")
    df[["subway", "links", "destination_from_nearest_subway"]] = df[
        ["subway", "links", "destination_from_nearest_subway"]].astype(str)
    df.links = df.links.str.replace("[", "")
    df.links = df.links.str.replace("]", "")
    df.links = df.links.str.replace("'","")

    df["quantity_of_metres"] = (df["full_price"] / df["square_price"]).round()
    df["date_of_parsing"] = datetime.today().strftime('%Y-%m-%d')
    df["date_of_parsing"] = pd.to_datetime(df["date_of_parsing"])
    return df

In [ ]:
full_df_test = filter_df(full_df_test)
full_df_test.head()

In [ ]:
def uncommon_values(df_new):

    # We extract all records from the table and create a new dataframe:
    df_old = client.execute("Select * from avito_flats")
    df_old = pd.DataFrame(columns=["square_price", "full_price", "subway", "links", "destination_from_nearest_subway", "quantity_of_metres", "date_of_parsing"],
                          data=df_old)
    df_new_values = df_new.merge(df_old,on="links",how="left",indicator=True).loc[lambda x: x["_merge"] == 'left_only']
    df_new_values = df_new_values.drop(["square_price_y","full_price_y","subway_y","destination_from_nearest_subway_y","quantity_of_metres_y", "date_of_parsing_y", "_merge"],axis=1)
    df_new_values = df_new_values.rename(columns={"square_price_x":"square_price", "full_price_x":"full_price","subway_x":"subway","destination_from_nearest_subway_x":"destination_from_nearest_subway","quantity_of_metres_x":"quantity_of_metres","date_of_parsing_x":"date_of_parsing"})

    return df_new_values

In [ ]:
full_df_new_values = uncommon_values(full_df_test)
full_df_new_values.head()

Now it's worth checking that everything works, and this can be done using inner join, because as we know, it searches for common elements, so if it returns an empty data frame, then the selected data frame contains only new values:

In [ ]:
click_data = client.execute("Select * from avito_flats")
click_data = pd.DataFrame(columns=["square_price", "full_price", "subway", "links", "destination_from_nearest_subway", "quantity_of_metres", "date_of_parsing"],
                      data=click_data)

In [ ]:
full_df_new_values.merge(click_data,how="inner",on="links")

We made sure that everything is fine and now we can continue, but before that we should immediately add new data to the database:

In [ ]:
def to_clickhouse(df):
    client.execute("INSERT INTO default.avito_flats VALUES", df.to_dict(orient="records"))
    client.execute('OPTIMIZE TABLE avito_flats DEDUPLICATE BY links')

In [ ]:
to_clickhouse(full_df_new_values)

How will cheap apartments be selected?
- The approach is as follows: first, we will look at the value of the 15th quantile at each metro station and then filter all values by it, thereby being able to select cheap apartments by metro station. I will also take apartments that are within 10 minutes of the metro.

In [ ]:
def cheap_flats(df):
    def q15(x):
        return x.quantile(0.15)

    # I create a data frame for each station with the 10th percentile for each metric
    metro_10_percentile = df.groupby("subway", as_index=False).agg({"full_price": q15, "square_price": q15}).rename(
        columns={"full_price": "full_price_10_percentile", "square_price": "square_price_10_percentile"}).sort_values(
        "square_price_10_percentile", ascending=False)

    needed_flats = df.query("destination_from_nearest_subway in ('5','6-10')")
    both_frames = needed_flats.merge(metro_10_percentile, how="left", on="subway")
    cheep_flats = both_frames.query(
        "square_price <= square_price_10_percentile and full_price <= full_price_10_percentile").drop_duplicates(
        subset="links")
    cheep_flats = cheep_flats[
        ["square_price", "full_price", "subway", "links", "destination_from_nearest_subway", "quantity_of_metres"]]

    return cheep_flats

In [ ]:
needed_df = cheap_flats(full_df_test)
needed_df.head()

In [ ]:
def send_cheap_flats(df):
    for index, row in df.iterrows():
        sleep(7)
        current_row = f"A great apartment deal just appeared on the website:\n" \
                      f"- Price per square meter: {row[0]};\n" \
                      f"- Total price = {row[1]};\n" \
                      f"- Area = {row[5]} square meters;\n" \
                      f"- Located near metro station = {row[2]} ({row[4]} minutes from the metro).\n" \
                      f"If you're interested in this offer, follow the link: {row[3]}"
        bot.sendMessage(chat_id=chat_id, text=current_row)

In [ ]:
send_cheap_flats(needed_df)